# <span style="color:blue"> SLACK - BOT</span>

In [2]:
# Importamos librerías necesarias
import pandas as pd
import numpy as np
from textblob import TextBlob
from datetime import datetime
import time
import json
from hashlib import md5

## <span style="color:blue"> 1) Exploración de los datos y hasheo </span>

In [10]:
# Nos permite trabajar con el dataset local en caso de teenrlo,
# Sino nos permite obtenerlo desde la url donde está alojado.

import os 
filename = "files/slack_historial_de_mensajes.csv" if os.path.exists("files/slack_historial_de_mensajes.csv") else "https://teampulse-bot.s3.amazonaws.com/slack_historial_de_mensajes.csv"
df = pd.read_csv(filename,index_col=0)

In [11]:
df.columns

Index(['client_msg_id', 'type', 'text', 'user', 'ts', 'team', 'blocks',
       'bot_id', 'bot_profile', 'files', 'upload', 'display_as_bot',
       'thread_ts', 'reply_count', 'reply_users_count', 'latest_reply',
       'reply_users', 'subscribed', 'reactions', 'subtype', 'inviter',
       'edited', 'attachments', 'Channel_id', 'Channel_name', 'bot_link',
       'purpose'],
      dtype='object')

In [12]:
df.shape

(784, 27)

### Generación de Hash para los usuarios y eliminacion de la columna USER

In [5]:
# Es necesario setear el encoding utf-8 andes de hashear
# Los floats no tienen atributo 'encode' por eso los debemos pasar a str

def hashit(val):
    if isinstance(val, float): 
        return str(val)
    return md5(val.encode('utf-8')).hexdigest()


df['user_hash']=df['user'].apply(hashit)

In [6]:
df=df.drop(['user'], axis=1)

In [7]:
df.shape

(784, 28)

In [8]:
df.columns

Index(['Unnamed: 0', 'client_msg_id', 'type', 'text', 'ts', 'team', 'blocks',
       'bot_id', 'bot_profile', 'files', 'upload', 'display_as_bot',
       'thread_ts', 'reply_count', 'reply_users_count', 'latest_reply',
       'reply_users', 'subscribed', 'reactions', 'subtype', 'inviter',
       'edited', 'attachments', 'Channel_id', 'Channel_name', 'bot_link',
       'purpose', 'user_hash'],
      dtype='object')

## <span style="color:blue">2) Filtrado de los mensajes irrelevantes </span>

In [9]:
# 1) Filtramos los mensajes que haya publicado el bot
df= df.drop(df[df['bot_id']=='B01CM1J688G'].index)

In [10]:
df.shape

(564, 28)

In [11]:
# 2) Filtramos los mensajes de aviso de un nuevo miembro en el canal
df.drop(df[df.text.str.contains('se ha unido al canal', na=False)].index, inplace = True)

In [12]:
df.shape

(529, 28)

In [13]:
# 3) Filtramos los mensajes de pedidos de descarga para el bot
df.drop(df[df.text.str.contains('descargar', na=False)].index, inplace = True)

In [14]:
df.shape

(456, 28)

In [15]:
# 4) Filtramos los mensajes en los que arrobamos al bot para saludarlo
df.drop(df[df.text.str.contains('<@U01BXC8LK27>', na=False)].index, inplace = True)

In [16]:
df.shape

(433, 28)

In [17]:
# 5) Filtramos los mensajes en los que se comparten direcciones web
df.drop(df[df.text.str.contains('<https://', na=False)].index, inplace = True)

In [18]:
df.shape

(397, 28)

In [19]:
# 5) Filtramos los mensajes nulos NaN
df=df.dropna(subset=['text'])

In [20]:
df.shape

(378, 28)

In [21]:
# Despues del proceso de filtrado, reseteamos el índice para evitar problemas futuros
df.reset_index(inplace=True)

## <span style="color:blue">3) Transformación de datos</span>

### 3.1) Transformación de fechas
* La variable fecha viene en formato timestamp (marca de tiempo Unix): Cantidad de segundos que han transcurrido desde el **01/01/1970**

In [22]:
# Transformo la variable 'ts'(timestamp) a una nueva variable 'datetime'
date = []

for i in range(0, len(df)):
    timestamp=df.iloc[i]['ts']
    dt_object = datetime.fromtimestamp(timestamp)
    date.append(dt_object)
df['datetime'] = pd.Series(date)

In [23]:
# Separo la información de fecha y hora en columnas diferentes
df['date'] = [d.date() for d in df['datetime']]
df['time'] = [d.time() for d in df['datetime']]

In [24]:
# Separo la información de hora, minuto y segundo en columnas diferentes
df[["h","m","s"]] = pd.DataFrame([(x.hour, x.minute, x.second) for x in df["time"]])

### 3.3) Traducción de los mensajes

In [25]:
message = TextBlob('hola buenos dias')
new = message.translate(from_lang='es', to='en')
str(new)

'Hello good day'

In [26]:
df['translations'] ="A"

In [27]:
error_message=""
numero=0

try:
    for i, row in df.iterrows():
        numero=i
        error_message = row['text']
        message = TextBlob(row['text'])
        if row['translations'] =="A":
            try:
                new = message.translate(from_lang='es', to='en')
                df.loc[i,'translations'] = str(new)
                time.sleep(3)
            except Exception as e:
                if (str(e))=="Translation API returned the input string unchanged.":
                    df.loc[i,'translations'] = row['text']
                else:
                    print(str(e))
            

except Exception as e: 
    print("An error occurred")
    print(error_message)
    print(str(numero))
    print(str(e))

In [28]:
df['translations'][:15]

0     Thank you. I already wrote, I hope someone wil...
1                                     Take care no more
2                            Hello good day ! ok: wink:
3                                      Nothing happens!
4                                    Go Ivan! Good day!
5     Hi. Very good days.\nI won't be able to be in ...
6                                                Hahaha
7     If you can close this slack for today so I don...
8     haha it could be ... I promise you I don't hav...
9     Vero, you are going to have to consider anothe...
10                       I've been moving forward there
11       But I already have something for that in sight
12                               I never command things
13                                                   No
14    You need to be unified in order to choose who ...
Name: translations, dtype: object

In [29]:
df['text'][:15]

0       Gracias. Ya les escribí, espero se una alguien.
1                                        Cuidese no mas
2                          hola buenos días ! ok :wink:
3                                         No pasa nada!
4                                  Dale Ivan! Buen día!
5     Hola. Muy buenos días.\nNo podré estar hoy en ...
6                                            Jajajajaja
7     Si pueden por hoy cerrar este slack asi no les...
8     jaja puede ser... les prometo que no me queda ...
9     Vero, te vas a tener que plantear otro modo de...
10                                  Vengo avanzando ahi
11                 Pero ya tengo en vista algo para eso
12                             Nunca me mando las cosas
13                                                   No
14    Necesitas que este unificado para poder elegir...
Name: text, dtype: object

In [31]:
# Lo guardamos como un archivo .csv
df.to_csv('files/slack_historial_de_mensajes_filtrados.csv', sep=',')